In [2]:
import pandas as pd
import numpy as np
import duckdb

In [10]:
eu27_countries = [
    "AT", "BE", "BG", "HR", "CY", "CZ", 
    "DK", "EE", "FI", "FR", "DE", "GR", 
    "HU", "IE", "IT", "LV", "LT", "LU", 
    "MT", "NL", "PL", "PT", "RO", "SK", 
    "SI", "ES", "SE"
]
INDUSTRY_CLASSIFICATION_PATH = "../data_raw/firmographics/industry_classifications/*.parquet"
CONTACT_INFO_PATH = "../data_raw/firmographics/contact_info/*.parquet"
LEGAL_INFO_PATTERN = "../data_raw/firmographics/legal_info/*.parquet"


def fetch_data_for_country(country_iso_code):
    query = f"""
        SELECT 
            nace.bvd_id_number,
            nace.nace_rev_2_core_code_4_digits_,
            nuts.nuts2,
            legal_info.status,
            legal_info.type_of_entity
        FROM
            '{INDUSTRY_CLASSIFICATION_PATH}' AS nace
        LEFT JOIN
            '{CONTACT_INFO_PATH}' AS nuts
        ON
            nace.bvd_id_number = nuts.bvd_id_number
        LEFT JOIN
            '{LEGAL_INFO_PATTERN}' AS legal_info
        ON
            nace.bvd_id_number = legal_info.bvd_id_number
        WHERE
            nace.bvd_id_number IS NOT NULL
        AND 
            (nace.nace_rev_2_core_code_4_digits_ IS NOT NULL 
            OR nuts.nuts2 IS NOT NULL 
            OR legal_info.status IS NOT NULL 
            OR legal_info.type_of_entity IS NOT NULL)
        AND
            nuts.country_iso_code = '{country_iso_code}'
    """

    # Connect to DuckDB and execute the query
    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    
    return df



In [ ]:
# country = "IT"
# conn = duckdb.connect()

# df = fetch_data_for_country(country)


In [ ]:
output_path = "..\\data_processed\\firmographics_processed\\"

for country in eu27_countries:
    df = fetch_data_for_country(country)
    df.to_parquet(f"{output_path}firmographics_{country}.parquet")
    print(f"{country}")

,bvd_id_number,nace_rev_2_core_code_4_digits_,nuts2,status,type_of_entity
0,IT03963730043,2059,ITC1 - Piemonte,Active,Corporate
1,IT03963730274,None,ITH3 - Veneto,Active,Corporate
2,IT03963730365,1089,ITH5 - Emilia-Romagna,Active,Corporate
3,IT03929420929,None,ITG2 - Sardegna,Active,Corporate
4,IT03966321212,None,ITF3 - Campania,Status unknown,Corporate
...,...,...,...,...,...
14474235,IT00840800536,None,ITI1 - Toscana,Active,Corporate
14474236,IT01603780196,5610,ITC4 - Lombardia,Active,Corporate
14474237,IT00911730174,4764,ITC4 - Lombardia,Active,Corporate
14474238,IT00840800536,None,ITI1 - Toscana,Active,Corporate


In [11]:
country = "IT"
conn = duckdb.connect()
df = conn.execute().fetchdf()
df

TypeError: execute(): incompatible function arguments. The following argument types are supported:
    1. (self: duckdb.duckdb.DuckDBPyConnection, query: object, parameters: object = None) -> duckdb.duckdb.DuckDBPyConnection

Invoked with: <duckdb.duckdb.DuckDBPyConnection object at 0x000001E2B22AFBB0>

In [7]:
df.to_parquet("E:/dati_moody/data_processed/additional_data.parquet", index=False)

In [4]:
file_pattern = "firmografics/industry_classifications/*.parquet"

columns = 'bvd_id_number, nace_rev_2_core_code_4_digits_'

query = f"""
    SELECT
        {columns}
    FROM
        '{file_pattern}'
    WHERE
        nace_rev_2_core_code_4_digits_ IS NOT NULL
"""

conn = duckdb.connect()


In [ ]:
import os
os.listdir("data_processced")

In [5]:
df.to_parquet('data_processed/nace.parquet', index=False)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

## Nuts

In [3]:


# Use glob to get all files in the contact_info folder with full path
file_pattern = "firmografics/contact_info/*.parquet"

columns = 'bvd_id_number, country_iso_code, nuts2, latitude, longitude'

query = f"""
    SELECT 
        {columns}
    FROM 
        '{file_pattern}'
    WHERE
        "nuts2" IS NOT NULL
        OR "latitude" IS NOT NULL
        OR "longitude" IS NOT NULL
"""


conn = duckdb.connect()
df = conn.execute(query).fetchdf()


In [4]:
df['latitude'] = df['latitude'].apply(dms_to_decimal)
df['longitude'] = df['longitude'].apply(dms_to_decimal)

In [5]:
df.to_parquet("E:/dati_moody/data_processed/nuts2.parquet", index=False)

## merge data

In [ ]:
nuts = pd.read_parquet("E:/dati_moody/data_processed/nuts2.parquet")
nace = pd.read_parquet("E:/dati_moody/data_processed/nace.parquet")

In [ ]:
merged_data = pd.merge(nuts, nace, on='bvd_id_number', how='inner')
merged_data

In [ ]:
merged_data.to_parquet("E:/dati_moody/data_processed/additional_data.parquet", index=False)

In [55]:
PATH = "E:\dati_moody\firmografics\contact_info"
PATH = "firmografics/contact_info/part-02253-54d921cc-a1c4-4ba5-941f-d71e06e76b2d-c000.snappy.parquet"

In [56]:
columns = 'bvd_id_number, country_iso_code, latitude, longitude'

In [57]:
country = 'IT'

In [92]:
# query = f"""
#     SELECT *
#     FROM 
#         '{PATH}'
#     WHERE
#         "latitude" IS NOT NULL
#         AND "longitude" IS NOT NULL
# """

# conn = duckdb.connect()
# df = conn.execute(query).fetchdf()

# df['longitude'] = df['longitude'].apply(dms_to_decimal)
# df['latitude'] = df['latitude'].apply(dms_to_decimal)

# df

In [93]:
# df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs={'init': 'epsg:4326'})
# df = df.to_crs({'init' :'epsg:4326'})

In [94]:
# # omi = gpd.GeoDataFrame.from_file("E:\\dati_moody\\nuts\\polygons\\NUTS_RG_20M_2024_4326.shp", driver='SHP')

# omi = gpd.GeoDataFrame.from_file("E:\\dati_moody\\nuts\\points\\NUTS_LB_2024_4326.shp", driver='SHP')
# omi = omi.to_crs({'init' :'epsg:4326'})

# omi['latitude'] = omi.geometry.y
# omi['longitude'] = omi.geometry.x

# # omi = omi[['NUTS_ID', 'latitude', 'longitude']]
# omi

In [ ]:
# # Filtering the DataFrame based on the conditions for latitude and longitude
# filtered_df = omi.loc[
#     (omi['latitude'] > 43) & (omi['latitude'] < 44) &
#     (omi['longitude'] > 10) & (omi['longitude'] < 11)
# ]
# filtered_df
